In [1]:
import cPickle as pickle
import numpy as np
import pandas as pd

from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer

In [2]:


typesinput = {
    'itemID': np.dtype(int),
    'categoryID': np.dtype(int),
    'title': np.dtype(str),
    'description': np.dtype(str),
    'images_array': np.dtype(str),
    'attrsJSON': np.dtype(str),
    'price': np.dtype(float),
    'locationID': np.dtype(int),
    'metroID': np.dtype(float),
    'lat': np.dtype(float),
    'lon': np.dtype(float),
}


# DESCRIPTION TF-IDFS

In [ ]:

items = pd.read_csv("input/ItemInfo_train.csv", dtype=typesinput, usecols=['description'])
itemstest = pd.read_csv("input/ItemInfo_test.csv", dtype=typesinput, usecols=['description'])
items.fillna("", inplace=True)
itemstest.fillna("", inplace=True)

In [ ]:

# Sprawdz tf-idf bez stopwordsow
tfidf = TfidfVectorizer(stop_words = stopwords.words('russian')).fit(pd.concat([items.description, itemstest.description]))

In [ ]:
transf = tfidf.transform(items.description)
pickle.dump(transf, open("tfidfs/description_stopwords_train.data","w"))

In [ ]:
transf = tfidf.transform(itemstest.description)
pickle.dump(transf, open("tfidfs/description_stopwords_test.data","w"))

# TITLE TF-IDFS

In [ ]:

items = pd.read_csv("input/ItemInfo_train.csv", dtype=typesinput, usecols=['title'])
itemstest = pd.read_csv("input/ItemInfo_test.csv", dtype=typesinput, usecols=['title'])
items.fillna("", inplace=True)
itemstest.fillna("", inplace=True)

In [ ]:

tfidf = TfidfVectorizer(stop_words = stopwords.words('russian')).fit(pd.concat([items.title, itemstest.title]))
transf = tfidf.transform(items.title)
pickle.dump(transf, open("tfidfs/title_stopwords_train.data","w"))
transf = tfidf.transform(itemstest.title)
pickle.dump(transf, open("tfidfs/title_stopwords_test.data","w"))

In [ ]:

tfidf = TfidfVectorizer().fit(pd.concat([items.title, itemstest.title]))
transf = tfidf.transform(items.title)
pickle.dump(transf, open("tfidfs/title_train.data","w"))
transf = tfidf.transform(itemstest.title)
pickle.dump(transf, open("tfidfs/title_test.data","w"))

# COSINE 

In [3]:
types1 = {
        'itemID_1': np.dtype(int),
        'itemID_2': np.dtype(int),
        'isDuplicate': np.dtype(int),
        'generationMethod': np.dtype(int),
}

pairs = pd.read_csv("input/ItemPairs_test.csv", dtype=types1)

In [4]:
tfidfs_desc = pickle.load(open("tfidfs/description_stopwords_test.data","r"))

In [5]:
tfidfs_desc.shape

(1315205, 1363634)

In [6]:
itemstest = pd.read_csv("input/ItemInfo_test.csv", dtype=typesinput, usecols=['itemID'])

In [7]:
itemstest.head()

,itemID
0,5
1,6
2,11
3,23
4,30


In [8]:
pairs.head()

,id,itemID_1,itemID_2
0,0,5,4670875
1,1,5,787210
2,2,6,1705280
3,3,11,3020777
4,4,23,5316130


In [9]:
index_ = pd.Index(itemstest.itemID)
pairs.itemID_1 = pairs.apply(lambda x: index_.get_loc(x['itemID_1']), axis=1)

In [10]:
pairs.itemID_2 = pairs.apply(lambda x: index_.get_loc(x['itemID_2']), axis=1)

In [ ]:
# Indeksy przerobione, liczymy podobieństwo

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity as cosine
pairs['description_cosine_sim'] = pairs.apply(lambda x: cosine(tfidfs_desc[x['itemID_1']],
                                                               tfidfs_desc[x['itemID_2']]), axis=1)